In [33]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset 
import cv2
import os


import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import SubsetRandomSampler, ConcatDataset
from torch.utils.data import random_split

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 
from torch.utils.data import Dataset 

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import albumentations as A 
from albumentations.pytorch import ToTensorV2 
import time

from einops import rearrange, repeat, reduce
from einops.layers.torch import Rearrange

ModuleNotFoundError: No module named 'einops'

In [2]:
print(os.getcwd())

path = os.path.join(os.getcwd(),"Dataset")
parse = ["alignments", "s1","s10"]
trans = A.Compose( [A.Normalize( 
            mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255
        ), ToTensorV2()])

labels = []
vids = []
tags = []
tokens = []

image_size = 32

for i in parse:
    p = os.path.join(path,i)
    if i == "alignments":
        for j in ["s1","s10"]:
            for al in os.listdir(os.path.join(p,j)):
                filePath=os.path.join(p,j,al)
                token = []
                tag = al[:-6]

                with open(filePath) as file:
                    for l in file:
                        l = l.strip()
                        l = l.split(" ")
                        if l[-1] == "sil" or l[-1] == "sp":
                            continue
                        else:
                            token.append(l[-1])
                            if l[-1] not in tokens:
                                tokens.append(l[-1])
                tags.append(tag)
                labels.append((token,tag, j))
    else:
        for j in os.listdir(p):
            if j[:-4] in tags:
                frames = []
                cap = cv2.VideoCapture(os.path.join(p,j))
                while(cap.isOpened()):
                    ret, frame = cap.read()
                    if ret == True:
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        frame = cv2.resize(frame, (image_size,image_size))
                        frame = trans(image=frame)["image"]
                        frames.append(frame)
                    else:
                        break
                vid = np.stack(frames, axis=0)
                cap.release()
                vids.append((vid, j[:-4], i))
            else:
                continue


C:\Users\friez\OneDrive - Tilburg University\M_CSAI\Year_2_Sem_1\Deep AI\Project


In [3]:
dataset = pd.DataFrame()

tokens = set(tokens)
tokens=list(tokens)
tokens.sort()
print(tokens)
conver = list(range(len(tokens)))
print(conver)


for lab, tag, part in labels:
    for vid, tag2, part2 in vids:
        if part != part2:
            continue

        if tag == tag2:
            newLab = []
            for word in lab:
                newLab.append(tokens.index(word)+2)
            newLab.append(1)
            if len(lab) == 7:
                print(lab, tag, part)
            frames = [vid, lab, newLab]
            dataset=pd.concat([dataset,pd.Series(frames).to_frame().T],ignore_index=True)

['a', 'again', 'at', 'b', 'bin', 'blue', 'by', 'c', 'd', 'e', 'eight', 'f', 'five', 'four', 'g', 'green', 'h', 'i', 'in', 'j', 'k', 'l', 'lay', 'm', 'n', 'nine', 'now', 'o', 'one', 'p', 'place', 'please', 'q', 'r', 'red', 's', 'set', 'seven', 'six', 'soon', 't', 'three', 'two', 'u', 'v', 'white', 'with', 'x', 'y', 'z', 'zero']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [4]:
dataset = dataset.rename(columns ={0: "Video" ,1: "Label", 2: "NumLabel"})
dataset.head()

,Video,Label,NumLabel
0,[[[[0.3254902 0.3921569 0.42352945 0.4470588...,"[bin, blue, at, f, two, now]","[6, 7, 4, 13, 44, 28, 1]"
1,[[[[0.3137255 0.3529412 0.4156863 0.4470588...,"[bin, blue, at, f, three, soon]","[6, 7, 4, 13, 43, 41, 1]"
2,[[[[0.2901961 0.36862746 0.40784317 0.4470588...,"[bin, blue, at, f, four, please]","[6, 7, 4, 13, 15, 33, 1]"
3,[[[[0.30588236 0.34901962 0.41176474 0.4313725...,"[bin, blue, at, f, five, again]","[6, 7, 4, 13, 14, 3, 1]"
4,[[[[0.3137255 0.3529412 0.4156863 0.4313725...,"[bin, blue, at, l, six, now]","[6, 7, 4, 23, 40, 28, 1]"


In [5]:
image = dataset["Video"].iloc[0]
print(image.shape)

(75, 3, 32, 32)


In [6]:
class MyDataset(Dataset):
 
    def __init__(self,df):
        df=df
 
        x=df.iloc[:,0].values
        y=df.iloc[:,2].values
        


        
        
 
        self.x_train=x
        self.y_train=y


 
    def __len__(self):
        return len(self.y_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]
    
    def getX(self):
        return self.x_train
    
    def getY(self):
        return self.y_train

In [7]:
train_data,val_data = train_test_split(dataset, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(val_data,test_size=0.5, random_state=42)

In [8]:
batch_size = 64
MAX_LENGTH = 6


train_data = MyDataset(train_data)
val_data = MyDataset(val_data)
test_data = MyDataset(test_data)

train_dl = DataLoader(train_data, batch_size)
val_dl = DataLoader(val_data, batch_size)
test_dl = DataLoader(test_data,batch_size)


In [20]:
SOS_token = 0
EOS_token = 1

class FeatureEmbedder(nn.Module):
    
    def __init__(self, d_feat, d_model):
        super(FeatureEmbedder, self).__init__()
        self.d_model = d_model
        self.embedder = nn.Linear(d_feat, d_model)
        
    def forward(self, x): # x - tokens (B, seq_len, d_feat)
        x = self.embedder(x)
        x = x * np.sqrt(self.d_model)
        
        return x # (B, seq_len, d_model)
    
class PositionalEncoder(nn.Module):
    
    def __init__(self, d_model, dout_p, seq_len=3660): # 3651 max feat len for c3d
        super(PositionalEncoder, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dout_p)
        
        pos_enc_mat = np.zeros((seq_len, d_model))
        odds = np.arange(0, d_model, 2)
        evens = np.arange(1, d_model, 2)

        for pos in range(seq_len):
            pos_enc_mat[pos, odds] = np.sin(pos / (10000 ** (odds / d_model)))
            pos_enc_mat[pos, evens] = np.cos(pos / (10000 ** (evens / d_model)))
        
        self.pos_enc_mat = torch.from_numpy(pos_enc_mat).unsqueeze(0)
        
    def forward(self, x): # x - embeddings (B, seq_len, d_model)
        B, S, d_model = x.shape
        # torch.cuda.FloatTensor torch.FloatTensor
        x = x + self.pos_enc_mat[:, :S, :].type_as(x)
        x = self.dropout(x)
        
        return x # same as inpu
    
class Encoder(nn.Module):
    
    def __init__(self, d_model, dout_p, H, d_ff, N):
        super(Encoder, self).__init__()
        self.enc_layers = clone(EncoderLayer(d_model, dout_p, H, d_ff), N)
        
    def forward(self, x, src_mask): # x - (B, seq_len, d_model) src_mask (B, 1, S)
        for layer in self.enc_layers:
            x = layer(x, src_mask)
        
        return x # x - (B, seq_len, d_model) which will be used as Q and K in decoder
    

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        patch = self.patch(input)
        embedded = self.dropout(self.embedding(patch))
        output, hidden = self.gru(embedded)
        return output, hidden
    
class AddAttention(nn.Module):
    def __init__(self, hidden_size):
        super(AddAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = AddAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [24]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

def tensorFromSentence(lang, indexes):
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [26]:
torch.cuda.empty_cache()
hidden_size = 128
device = "cuda" if torch.cuda.is_available() else "cpu"

encoder = EncoderRNN(3, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, len(tokens)+2).to(device)

train(train_dl, encoder, decoder, 80, print_every=5, plot_every=5)

torch.Size([64, 75, 3, 32, 32])


RuntimeError: Given groups=1, weight of size [96, 3, 2, 4, 4], expected input[64, 75, 4, 32, 32] to have 3 channels, but got 75 channels instead